## 영상 특징 검출

1. 영상으로부터 계산할 수 있는 영상 특징은 매우 다양하다. 이미 설명한 에지(edges), 직선(lines), 원(circles)을 포함한 코너점(corner points), 사각형(rectangle) 등의 구조적인 특징이있다.
2. 영상의 밝기/컬러의 평균(averages), 분산(variances), 히스토그램(histograms), 분포(distribution), 그래디언트(gradients)의 크기 및 방향 등의 화소값 관련된 특징이 있다.
3. 이러한 영상 특징은 화소 주변 이웃으로부터 계산하는 지역 특징(local features)과 영상 전체에서 계산하는 전역 특징(global features)이 있다.
4. 특징 검출기(feature detector)는 영상으로부터 관심 영역의 위치(location)를 검출하는 알고리즘이다.
4. 특징 디스크립터(feature descriptors)는 영상 매칭(matching)을 위한 정보를 표현한 특징 벡터이다.

### 코너점 검출

- cv2.preCornerDetect(src, ksize) -> dst

##### 영상 src에서 코너점 검출을 위한 특징맵 dst를 Sobel 미분 연산자를 이용하여 계산한다. ksize는 Sobel 연산자의 마스크 크기이다.
##### 코너점은 dst에서 지역 극값에서 검출된다.

- cv2.cornerEigenValsAndVecs(src, blockSize, ksize) -> dst

##### 입력 영상 src에서 각 화소의 고유값과 고유 벡터를 6-채널 dst에 계산한다.
##### 영상의 모든 화소에 대하여, blockSize x blockSize의 이웃에 있는 미분 값을 이용하여 2 x 2 크기의 그래디언트를 이용한 공분산 행렬 M을 계산하고, M의 고유값 λ<sub>1</sub>, λ<sub>2</sub>, 고유 벡터 (x<sub>1</sub>, y<sub>1</sub>), (x<sub>2</sub>, y<sub>2</sub>)을 계산하여 dst에 저장한다.
##### 고유값 λ<sub>1</sub>, λ<sub>2</sub>이 모두 작은 곳은 평평한 영역에 있는 점이며, 고유값 λ<sub>1</sub>, λ<sub>2</sub> 중에서 하나는 크고 하나는 작으면 에지(edges)이며, 두 고유값 λ<sub>1</sub>, λ<sub>2</sub>이 모두 큰 곳이 코너점이다.

- cv2.cornerMinEigenVal(src, blockSize) -> dst

##### 입력 영상 src에서 각 화소의 최소 고유값을 dst에 계산한다.
##### 공분한 행렬 M으로부터 최소 고유값 min(λ<sub>1</sub>, λ<sub>2</sub>)을 출력 행렬 dst에 저장한다.
##### dst에서 임계값보다 큰 화소가 코너점이다.

- cv2.cornerHarris(src, blockSize, ksize, k) -> dst

##### 입력 영상 src에서 각 화소의 Harris 반응값을 dst에 계산한다.
##### k는 Harris 코너 검출 상수로 0.01에서 0.06사이의 값을 주로 사용한다.
##### Harris 코너 검출 반응값의 행렬 dst에서 지역 극대값이 코너점이 된다.
##### dst(x, y) = det(M(x, y)) - k * trace(M(x, y))<sup>2</sup>

- cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria) -> corners

##### 입력 영상 image에서 검출된 코너점 corners를 입력하여 코너점의 위치를 부화소 수준으로 다시 계산하여 반환한다.
##### winSize은 탐색영역의 크기를 정의하며, 예를 들어 winSize = (3, 3)이면, 탐색 영역을 (3 x 2 + 1) x (3 x 2 + 1)크기이다.
##### zeroZone을 설정하면 winSize 영역 내에서 해당 영역을 마스크 처리하여 탐색 영역에서 계산하지 않는다.
##### zeroZone = (-1, -1)이면 zeroZone이 설정되지 않는다. criteria는 최대 반복회수와 오차를 사용한 종료 조건이다.

- cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k ]]]]]) -> corners

##### 영상 image에서 추적하기 좋은 강한 코너점을 검출한다.
##### maxCorners는 최대 코너점 개수이고, qualityLevel는 최소 코너점의 질(quality)을 결정하는 값이다.
##### minDistance는 코너점들 사이의 최소 거리이다.
##### mask는 코너점이 검출될 영역을 지정하며, mask = None이면 영상 전체에서 코너점을 계산한다.
##### blockSize는 블록의 크기이고, useHarrisDetector = False이면 cv2.cornerMinEigenVal()를 사용하고, useHarrisDetector = True이면 cv2.cornerHarris()를 사용한다.
##### k는 해리스 코너점 검출에 사용되는 상수이다.

#### 실습

In [25]:
## cv2.preConerDetect() 코너점 검출
# 0801.py
import cv2
import numpy as np
#1
def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape = cv2.MORPH_RECT, ksize = (11, 11))
    # local max if kernel = None, 3x3
    dilate = cv2.dilate(src, kernel)
    localMax = (src == dilate)
    # lacal min if kernel = None, 3x3
    erode = cv2.erode(src, kernel)
    localMax2 = src > erode
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0,1]] = points[:,[1,0]]
    # switch x, y
    return points
# findLocalmaxima()는 src에서 팽창과 침식의 모폴로지 연산으로 지역 극대값의 좌표흫 points 배열에 검출하여 반환한다.
# cv2.dilate()로 src에서 rectKernel의 이웃에서 최대값을 dilate에 계산한다.
# 커널을 None을 사용하면 3x3 사각형 이웃이다. src ==dilate로 src에서 지역 최대값의 위치를 localMax 배열에 계산한다.
# cv2.erode()로 src에서 rectKernel의 이웃에서 최소값을 erode에 계산한다.
# localMax2 = src > erode로 최소값보다 큰 위치를 localMax2에 계산한다. localMax &= localMax2로 localMax와 localMax2를 논리곱하여 지역 최대값 위치를 localMax 배열에 계산한다.
# points = np.argwhere()로 localMax 배열에서 True인 위치의 좌표를 points 배열에 찾는다.
# np.argwhere()는 행, 열 순서로 찾기 때문에, points[:, [0, 1]] = points[:, [1, 0]]에 의해 좌표순서를 열(x), 행(y)로 변경하여 반환한다.


#2
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize=3)
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO)
corners = findLocalMaxima(res2)
print('corners.shape=', corners.shape)
# 그레이스케일 영상 gray에서 cv2.preCornerDetect()로 res를 계산한다. 
# 극대값만을 찾기 위하여 np.abs(res)인 절대값 배열에서, cv2.threshold()로 임계값 thresh = 0.1보다 작은 값은 0으로 변경하여 res2에 저장한다.
# 즉, res에서 임계값보다 작은 값을 제거한다. findLocalMaxima()로 res2에서 지역 극값의 좌표를 코너점으로 찾아 corners에 저장한다.

#3
dst = src.copy()  
for x, y in corners:    
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)
# src를 dst에 복사하고, 코너점 배열 corners의 각 코너점 좌표에 cv2.circle()로 dst에 반지름 5, 빨간색 원을 그린다.
    
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


corners.shape= (8, 2)


In [34]:
# 코너점 검출: cornerEigenValsAndVecs()
# 0802.py
import cv2
import numpy as np
#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, blockSize=5, ksize=3)
print('res.shape=', res.shape)
eigen = cv2.split(res)
# cv2.cornerEigenValsAndVecs()로 gray 영상에서 각 화소 이웃에 대한 2x2 공분산 행렬 M의 고유값과 고유 벡터를 res에 계산한다.
# res.shape = (512, 512, 6)이다. cv2.split()로 res를 채널 분리하여 eigen에 저장한다.
# eigen[0] = λ₁, eigen[1] = λ₂의 고유값이고, λ₁에 대한 고유 벡터는 eigen[2] = x1, eigen[3] = y1이고, λ₂에 대한 고유 벡터는 eigen[4] = x2, eigen[5] = y2에 저장된다.

#2
T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)
# cv2.threshold()로 eigen[0]에서 임계값 T = 0.2로 이진 영상 edge를 검출한다.

#3
corners = np.argwhere(eigen[1]>T)
corners[:,[0, 1]] = corners[:,[1, 0]]
print('len(corners) =', len(corners))

dst = src.copy()
for x, y in corners:  
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)
# 작은 고유값 eigen[1]이 T보다 크면, 큰 고유값 eigen[0] T보다 크므로 np.argwhere()로 eigen[1] > T인 좌표를 코너점 배열 corners에 검출하고, corners[:, [0, 1]] = corners[:, [1, 0]]에 의해 좌표순서를 열 x, 행 y로 변경하여 반환한다.
# corners의 각 코너점 좌표에 cv2.circle()로 dst에 반지름 5인 빨간색 원을 표시한다.
    
cv2.imshow('edge',  edge) 
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


res.shape= (512, 512, 6)
len(corners) = 8


In [33]:
# 코너점 검출 1: cv2.cornerMinEigenVal()
# 0803.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, blockSize=5)
print('eigen.shape=', eigen.shape)
# cv2.cornerMinEigenVal()로 gray 영상에서 각 화소 이웃에 의한 2x2 공분산 행렬 M의 작은 고유값 λ₂를 eigen.shape (512, 512)인 eigen에 계산한다.

#2
T = 0.2
corners  = np.argwhere(eigen> T)
corners[:,[0, 1]] = corners[:,[1, 0]]
print('len(corners ) =', len(corners ))
dst = src.copy()
for x, y in corners :    
    cv2.circle(dst, (x, y), 3, (0,0,255), 2)
# np.argwhere()로 eigen > T인 좌표를 코너점 배열 corners에 검출하고, corners[:, [0,1]] = corners[:, [1, 0]]에 의해 좌표순서를 열x, 행y로 변경하여 반환한다.
# corners의 각 코너점 좌표에 cv2.circle()로 dst에 반지름 5인 빨간색 원을 표시한다.
    
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


eigen.shape= (512, 512)
len(corners ) = 8


In [32]:
# 코너점 검출 2: cv2.cornerHarris(), cv2.cornerSubPix()
# 0804.py
import cv2
import numpy as np

#1
def findLocalMaxima(src):
    kernel= cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(11,11))
    dilate = cv2.dilate(src,kernel)
    localMax = (src == dilate)
    
    erode = cv2.erode(src,kernel)
    localMax2 = src > erode      
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0, 1]] = points[:,[1, 0]] 
    return points
# findLocalMaxima()는 src에서 팽창과 침식의 모폴로지 연산으로 지역 극대값의 좌표를 points를 검출하여 반환한다.

#2
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.01)
ret, res = cv2.threshold(np.abs(res),0.02, 0, cv2.THRESH_TOZERO)
res8 = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imshow('res8',  res8)

corners = findLocalMaxima(res)
print('corners=', corners)
# cv2.cornerHarris()로 gray 영상에서 각 화소 이웃에 의한 2x2 공분산 행렬 M의 Harris 반응값을 res에 계산한다.
# res.shape = (512, 512)이다. cv2.threshold()로 np.abs(res)에서 임계값 thresh = 0.02보자 작은 값을 0으로 변경하여 res에 저장한다.
# Harris 반응값 res를 [0, 255]범위로 정규화한 res8은 다음과 같다.

#3
#corners = np.float32(corners).copy()
corners = corners.astype(np.float32, order='C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners,(5,5),(-1,-1), term_crit)
print('corners2=', corners2)

dst = src.copy()
for x, y in np.int32(corners2):    
    cv2.circle(dst, (x, y), 3, (0,0,255), 2)
# vindLocalMaxima()로 res에서 코너점을 찾아 corners에 저장한다. 이때의 코너점 corners는 정수 좌표이다.
# corners를 np.float32 자료형으로 변환한다. 이때 order = 'C'에 의해 C언어 스타일 메모리 구조를 지정하거나, 복사하지 않으면 cv2.cornerSubPix()에서 오류가 발생함에 주의한다.
# cv2.cornerSubPix()로 gray 영상에서 코너점 좌표 corners를 부화소 수준으로 계산하여 corners에 저장한다.
# 실행 결과를 보면 corners의 좌표는 corners의 좌표는 corners에서 약간 이동된 다른 것을 알 수 있다. corners2의 각 코너점 좌표에 cv2.circle()로 반지름 5인 빨간색 원을 표시한다.

cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


corners= [[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]
corners2= [[107.559364 125.559456]
 [265.44077  125.559235]
 [265.55923  168.44174 ]
 [387.4408   168.55888 ]
 [107.558365 269.44095 ]
 [168.4415   269.55927 ]
 [168.55922  375.4408  ]
 [387.4408   375.4408  ]]


In [40]:
# 코너점 검출 2: cv2.cornerHarris(), cv2.cornerSubPix()
# 0805.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.01)
# cv2.cornerHarris()로 gray 영상에서 각 화소 이웃에 의한 2x2 공분산 행렬 M의 Harris 반응값을 res에 계산한다.
# res.shape(512, 512)이다.

#2
res = cv2.dilate(res, None) # 3x3 rect kernel
ret, res = cv2.threshold(res, 0.01*res.max(),255,cv2.THRESH_BINARY)
res8 = np.uint8(res)
cv2.imshow('res8',  res8)
# cv2.dilate()로 res에 3x3 사각형 커널을 사용하여 팽창연산으로 지역 최대값을 res에 계산한다.
# cv2.threshold()로 res에서 임계값 thresh = 0.01 * res.max()보다 크면 255인 이진 영상을 res에 저장한다.
# Harris 반응값 res를 np.uin8 자료형으로 변경

#3
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(res8)
print('centroids.shape=', centroids.shape)
print('centroids=',centroids)
centroids = np.float32(centroids)
# cv2.connectedComponentsWithStats()로 이진 영상 res8를 레이블링하여 레이블개수 ret, 레이블 정보 labels, 통계정보 stats, 중심섬 centroids를 계산한다.
# 배경을 포함하기 때문에 ret = 9dlek. centroids의 자료형을 np.float32로 변경
# 반응값이 임계값보다 큰 영역의 중심인 centroids가 코너점이다. 이때는 order = 'C'를 지정하지 않아도 cv2.cornerSubPix()에서 오류가 발생하지 않는다.

#4
term_crit=(cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS,10, 0.001)
corners = cv2.cornerSubPix(gray, centroids, (5,5), (-1,-1), term_crit)
print('corners=',corners)
# cv2.cornerSubPix()로 gray 영상에서 centroids를 부화소 수준으로 계산하여 corners에 저장한다.
# corners[0]은 배경의 중심점이다. 물체의 코너점인 corners[1:]의 각 좌표에 cv2.circle()로 반지름 5인 빨간색 원을 표시한다.

#5
corners = np.round(corners)
dst = src.copy()
for x, y in corners[1:]:    
    cv2.circle(dst, (int(x), int(y)), 5, (0,0,255), 2)

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


centroids.shape= (9, 2)
centroids= [[255.53481922 255.53107522]
 [108.         126.        ]
 [265.         126.        ]
 [266.         168.        ]
 [387.         169.        ]
 [108.         269.        ]
 [168.         270.        ]
 [169.         375.        ]
 [387.         375.        ]]
corners= [[255.53482  255.53108 ]
 [107.55841  125.55865 ]
 [265.44165  125.55835 ]
 [265.55832  168.44258 ]
 [387.44168  168.55795 ]
 [107.557495 269.44186 ]
 [168.4424   269.55838 ]
 [168.55832  375.44168 ]
 [387.44168  375.44168 ]]


In [39]:
## cv2/goodFeaturesToTrack() 코너점 검출
# 0806.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

K = 5 
# K = 10
corners = cv2.goodFeaturesToTrack(gray, maxCorners=K,
              qualityLevel=0.05, minDistance=10)
print('corners.shape=',corners.shape)
print('corners=',corners)
# cv2.goodFeaturesToTrack()로 gray에서 최대 코너점 maxCorners = K를 적용하여 코너점 corners에 검출한다. K = 5인 경우, corners.shape = (5, 1, 2)로 5개의 코너점의 좌표가 (1, 2)에 저장된다.

#2
corners2 = cv2.goodFeaturesToTrack(gray, maxCorners=K,
               qualityLevel=0.05, minDistance=10,
               useHarrisDetector=True, k=0.04)
print('corners2.shape=',corners2.shape)
print('corners2=',corners2)
# cv2.goodFeaturesToTrack()로 gray에서 최대 코너점 maxCorners = K를 적용하여 코너점 corners2에 검출한다.
# K = 5인 경우, corner.shape = (5, 1, 2)로 5개의 코너점의 좌표가 (1, 2)에 저장된다.

#3
dst = src.copy()
corners = corners.reshape(-1, 2)
for x, y in corners:    
    cv2.circle(dst, (int(x), int(y)), 5, (0,0,255), -1)

corners2 = corners2.reshape(-1, 2)
for x, y in corners2:    
    cv2.circle(dst, (int(x), int(y)), 5, (255,0,0), 2)
    
# corners의 각 좌표에 cv2.circle()로 반지름 5인 빨간색 채워진 원으로 dst에 표시한다.
# corners2의 각 좌표에 cv2.circle()로 반지름 5인 파란색, 두께 2인 원으로 dst에 표시한다.
    
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


corners.shape= (5, 1, 2)
corners= [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[168. 270.]]

 [[266. 168.]]]
corners2.shape= (5, 1, 2)
corners2= [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[387. 169.]]

 [[108. 269.]]]


### 체스보드 패턴 코너점 검출

- cv2.findChessboardCorners(image, patternSize[, corners[, flags ]]) -> retval, corners

##### image에서 체스보드 패턴의 내부 코너점을 순차적으로 검출한다. 시작점은 왼쪽-위 또는 오른쪽-아래에서 시작하여 행우선 순서로 검출하여 corners에 반환한다.
##### image는 그레이스케일 또는 컬러 영상이고, patternSize는 패턴의 내부코너점의 열과 행의 크기로 patternSize = (points_per_row, points_per_colum)이다.
##### flags = 0이거나 cv2.CALIB_CB_ADAPTIVE_THRESH, cv2.CALIB_CB_NORMALIZE_IMAGE, cv2.CALIB_CB_FILTER_QUADS, cv2.CALIB_CB_FAST_CHECK를 조합해 사용한다.
##### 검출된 코너점은 CORNERS 배열에 반환한다.

- cv2.findCirclesGrid(image, patternSize[, centers[, flags[, blobDetector ]]]) -> retval, centers

##### 원 형태의 격자에서 원의 중심점을 검출한다.
##### patternSize는 패턴의 내부 코너점의 열과 행의 크기로 patternSize = (points_per_row, points_per_colum)이다.
##### flags는 cv2.CALIB_CB_SYMMETRIC_GRID, cv2.CALIB_CB_ASYMMETRIC_GRID, cv2.CALIB_CB_CLUSTERING 중 하나이다.
##### blobDetector는 사용할 blob 검출기이다.

- cv2.draChessboardCorners(image, patternSize, corners, patternWasFound) -> image

##### cv2.findChessboardCorners()로 검출된 코너점 배열 corners를 8비트 컬러 영상 image에 표시한다.
##### patternSize는 패턴의 크기, patternWasFound는 패턴이 발견되었는지 여부이다.

#### 실습

In [60]:
## 체스보드 패턴 코너점 검출: cv2.findChessboardCorners()
# 0807.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
found, corners = cv2.findChessboardCorners(src, patternSize = (6, 3))
print('corners.shape=', corners.shape)
# src영상은 7x4의 흰색과 검은색 사각형을 갖는다. 체스보드 패턴의 내부 코너점은 6x3이다.
# src 또는 gray에서 cv2.findChessboardCorners()로 patternSize(6,3)의 패턴 크기의 코너점을 corners에 검출한다.
# found = True이고, corners.shape = (18, 1, 2)으로 18개의 코너점의 좌표를 (1, 2)에 저장한다.

#2
term_crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), term_crit)
# cv2.cornerSubPix()로 corners를 부화소 수준으로 corners2에 계산한다.

#3
dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, corners2, found)
# src를 dst에 복사하고, dv2.drawChessboardCorners()로 검출된 corners2를 dst에 그려 표시한다.

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


corners.shape= (18, 1, 2)


In [61]:
# 원 패턴 중심점 검출: cv2.findCirclesGrid()
# 0808.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/circleGrid.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6, 4)
found, centers = cv2.findCirclesGrid(src, patternSize)
print('centers.shape=', centers.shape)
# src 영상은 6x4의 검은색 원 패턴을 갖는다. src 또는 gray에서 cv2.findCirclesGrid()로 patternSize(6, 4)의 원의 중심점을 centers에 검출한다.
# found = True이고, centers.shape = (24, 1, 2)으로 24개의 중심점의 좌표를 (1, 2)에 저장한다.

#2
term_crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
centers2 = cv2.cornerSubPix(gray, centers, (5,5), (-1,-1), term_crit)
# cv2.cornerSubPix()로 centers를 부화소 수준으로 centers2에 계산한다.

#3
dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, centers2, found)
# src를 dst에 복사하고, cv2.drawChessboardCorners()로 검출된 centers를 dst에 그려 표시한다.

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


centers.shape= (24, 1, 2)


### 모멘트 Moments

* 영상 모멘트는 화소의 가중평균으로 물체 인식을 위해 사용할 수 있는 디스크립터이다.
* 영상 분할하고 관심 물체로의 영상 모멘트를 계산하면, 물체의 면적, 무게중심, 물체의 기울어진 방향 등을 계산할 수 있다.

- cv2.moments(array[, binaryImage ]) -> retval

##### 경계선 좌표 또는 영상의 3-차 모멘트까지 계산한다.
##### array는 1-채널의 영상 또는 경계선 좌표 배열이다. binaryImage = True이면, array가 영상일 때 0이 아닌 화소값을 1로 처리한다.
##### 공간 모멘트, 중심 모멘트, 정규화 중심 모멘트를 계산하여 retval에 사전자료형으로 반환한다.
* 공간 모멘트
    * m<sub>ji</sub>는 공간 모멘트이다. j >= 0이고, i >= 0이고, i + j <= 3이다.
    * 영상 모멘트는 영상 화소값과 좌표를 이용하여 계산하고, 경계선 모멘트는 경계선 위의 좌표만은 가지고 모멘트를 계산한다.
    * m<sub>00</sub>은 이진 영상에서는 면적이고, 그레이스케일 영상에서는 밝기값의 합니다.
* 중심 모멘트
    * (x<sub>c</sub>,y<sub>c</sub>)는 무게중심이다. 중심 모멘트에서 mu00 = m00, mu10 = 0, mu01 = 0이다.
* 정규 중심 모멘트
    * 정규 중심 모멘트에서 nu00 = 1, nu10 = 0, nu01 = 0이다.

- cv2.HuMoments(m[, hu ]) -> hu

##### 정규 중심 모멘트를 이용하여 Hu의 7 모멘트를 계산한다.
##### m은 cv2.moments()로 계산한 모멘트이다.
##### Hu 모멘트는 이동, 스케일, 회전에 불변이다. 단, hu[6]은 영상 반사에 의해 부호가 변경된다.

#### 실습

In [65]:
## 영상 모멘트: cv2.moments()
# 0809.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/momentTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
# 입력 영상 src를 그레이스케일로 gray로 변환하고, 이진 영상 bImage를 생성한다.

#2
##M = cv2.moments(bImage)   
M = cv2.moments(bImage, True)
for key, value in M.items():
    print('{}={}'.format(key, value))
# cv2.moments()로 이진 영상 bImage에서 영상 모멘트 M을 계산한다.
# cv2.moments(bImage)는 bImage의 물체 영역의 화소값 255로 계산하고, cv2.moments(bImage, True)는 1로 계산하여 모멘트 값이 다르다.
# for문으로 M.items()의 key.value 값으로 모멘트를 출력한다.
    
#3
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
dst = src.copy()
cv2.circle(dst, (cx, cy), 5, (0,0,255), 2)
# 문체의 중심 좌표를 cx, cy에 계산하고, dst에 빨간색 원으로 표시한다.

cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


m00=79262.0
m10=19719561.0
m01=19943644.0
m20=5515429769.0
m11=5090506179.0
m02=5490383844.0
m30=1678594806585.0
m21=1448966367859.0
m12=1427069177889.0
m03=1619623824694.0
mu20=609408144.1012974
mu11=128735034.94252014
mu02=472229671.77704334
mu30=3184447093.48999
mu21=-2863822758.3268433
mu12=-3664976429.7761536
mu03=509725524.1086426
nu20=0.09700144427924574
nu11=0.020491167437841
nu02=0.07516630789606452
nu30=0.001800410227810075
nu21=-0.0016191368967214139
nu12=-0.002072090022265142
nu03=0.00028818662079742156


In [69]:
## cv2.moments() 경계선 모멘트
# 0810.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/circles.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
# 입력 영상 src를 그레이스케일로 gray로 변환하고, cv2.THRESH_BINARY_INV로 물체가 흰색, 배경이 검은색인 이진 영상 bImage를 생성한다.

#2
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
cv2.drawContours(dst, contours, -1, (255,0,0), 3)
# cv2.findContours()로 이진 영상 bImage에서 윤곽선(경계선)을 contours에 검출하고, dst에 파란색으로 윤곽선을 그린다.

#3
for cnt in contours:
    M = cv2.moments(cnt, True)
##    for key, value in M.items():
##        print('{}={}'.format(key, value))

    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.circle(dst, (cx, cy), 5, (0,0,255), 2)
# contours의 각 윤곽선 cnt의 모멘트를 M에 계산하고, 각 윤곽선의 중심좌표(cx, cy)를 계산하고, dst에 빨간색 원으로 표시한다.

cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


In [71]:
# 0811.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/momentTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
cnt = contours[0]
cv2.drawContours(dst, [cnt], 0, (255,0,0), 3)
# 입력 영상 src를 그레이스케일 gray로 변환하고, 이진 영상 bImage를 구하고, cv2.findContours()로 bImage에서 윤곽선을 contours에 검출하고,
# 첫 번째 윤곽선 contours[0]을 cnt에 저장하고 src를 복사한 dst에 파란색으로 윤곽선을 그린다.

#2
M = cv2.moments(cnt)
hu = cv2.HuMoments(M)
print('hu.shape=', hu.shape)
print('hu=', hu)
# cv2.moments()로 윤곽선 cnt의 경계선 모멘트 M을 계산하고, cv2.HuMoments()로 모멘트를 이용하여 Hu의 모멘트를 hu에 계산한다.

#3
angle = 45.0
scale = 0.2
cx = M['m10']/M['m00']
cy = M['m01']/M['m00']
center = (cx, cy)
t = (20, 30)
A = cv2.getRotationMatrix2D( center, angle, scale )
A[:, 2] += t
print('A=', A)

cnt2 = cv2.transform(cnt, A)
cv2.drawContours(dst, [cnt2], 0, (0,255,0), 3)
cv2.imshow('dst',  dst)

#4
M2 = cv2.moments(cnt2)
hu2 = cv2.HuMoments(M2)
print('hu2.shape=', hu2.shape)
print('hu2=', hu)

#5
##diffSum = sum(abs(hu - hu2))
diffSum = np.sum(cv2.absdiff(hu, hu2))
print('diffSum=', diffSum)

cv2.waitKey()
cv2.destroyAllWindows()


hu.shape= (7, 1)
hu= [[ 1.72272960e-01]
 [ 2.17960438e-03]
 [ 9.24428655e-05]
 [ 1.90785217e-06]
 [ 1.11977849e-12]
 [-6.96325160e-09]
 [-2.53121609e-11]]
A= [[ 1.41421356e-01  1.41421356e-01  1.98030817e+02]
 [-1.41421356e-01  1.41421356e-01  2.81234993e+02]]
hu2.shape= (7, 1)
hu2= [[ 1.72272960e-01]
 [ 2.17960438e-03]
 [ 9.24428655e-05]
 [ 1.90785217e-06]
 [ 1.11977849e-12]
 [-6.96325160e-09]
 [-2.53121609e-11]]
diffSum= 0.0003215707378081359
